In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import numpy as np
import os
import PIL
import PIL.Image

In [2]:
num_classes = 11 

img_height = 64
img_width = 64

model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [3]:
epochs = 25 # liczba epok treningu

data_dir = 'dataset'
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Found 6862 files belonging to 11 classes.
Using 5490 files for training.
Found 6862 files belonging to 11 classes.
Using 1372 files for validation.
Epoch 1/25
172/172 [==============================] - 21s 115ms/step - loss: 1.5180 - accuracy: 0.4862 - val_loss: 1.1821 - val_accuracy: 0.6006
Epoch 2/25
172/172 [==============================] - 20s 114ms/step - loss: 1.0612 - accuracy: 0.6412 - val_loss: 1.0635 - val_accuracy: 0.6480
Epoch 3/25
172/172 [==============================] - 20s 114ms/step - loss: 0.9494 - accuracy: 0.6747 - val_loss: 1.0494 - val_accuracy: 0.6531
Epoch 4/25
172/172 [==============================] - 18s 106ms/step - loss: 0.8699 - accuracy: 0.7018 - val_loss: 1.0128 - val_accuracy: 0.6560
Epoch 5/25
172/172 [==============================] - 19s 110ms/step - loss: 0.7880 - accuracy: 0.7319 - val_loss: 0.8955 - val_accuracy: 0.7048
Epoch 6/25
172/172 [==============================] - 17s 100ms/step - loss: 0.7212 - accuracy: 0.7557 - val_loss: 0.9271 - val

In [4]:
loss, accuracy = model.evaluate(val_ds)
print("Accuracy: ", accuracy)

43/43 [==============================] - 2s 30ms/step - loss: 1.7006 - accuracy: 0.7245
Accuracy:  0.7244898080825806


In [5]:
img_path = 'test/sandman.jpg'
class_names = ['dew', 'fogsmog', 'frost', 'glaze', 'hail', 'lightning', 'rain', 'rainbow', 'rime', 'sandstorm', 'snow']

img = keras.preprocessing.image.load_img(
    img_path, target_size=(img_height, img_width)
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "To zdjęcie najprawdopodobniej przedstawia {} z prawdopodobieństwem {:.2f}%"
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

1/1 [==============================] - 0s 206ms/step
To zdjęcie najprawdopodobniej przedstawia sandstorm z prawdopodobieństwem 100.00%
